In [3]:
import pandas as pd

In [ ]:
sku_dict = pd.read_csv('sku_dict.csv')

In [ ]:
sku_dict['SKU'] = sku_dict['sku_id'].tolist()

In [ ]:
groups = ['first_group_sku.csv', 'second_group_sku.csv', 'third_group_sku.csv']

In [ ]:
files = [
    'first_group_sku.csv',
    'second_group_sku.csv',
    'third_group_sku.csv'
]

### 1.1

In [ ]:
qnt_vendors = []
qnt_brands = []
for file in files:
    df = pd.read_csv(file)
    df = df.merge(sku_dict, how='left', on='SKU')
    qnt_vendors.append(len(set(df['vendor'].tolist())))
    qnt_brands.append(len(set(df['brand_code'].tolist())))

In [ ]:
df = pd.DataFrame()
df['file'] = [file[:-4] for file in files]
df['vendor_unique_number'] = qnt_vendors
df['brand_unique_number'] = qnt_brands
df.to_csv('unique_vendor_brand.csv', index=False)

### 1.2

In [ ]:
df_list = []
for file in files:
    df = pd.read_csv(file)
    df = df.merge(sku_dict, how='left', on='SKU')
    sku_list = []
    min_price_list = []
    max_price_list = []
    for s in set(df['SKU'].tolist()):
        prises = df[df['SKU'] == s]['price_per_sku'].tolist()
        min_price = min(prises)
        max_price = max(prises)
        sku_list.append(s)
        min_price_list.append(min_price)
        max_price_list.append(max_price)
    df = pd.DataFrame()
    df['SKU'] = sku_list
    df['min_price'] = min_price_list
    df['max_price'] = max_price_list
    df_list.append(df)
df = pd.concat(df_list)
df.to_csv('sku_min_max_prices.csv', index=False)

### 2

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('first_group_sku.csv')

In [ ]:
df = df.merge(sku_dict, how='left', on='SKU')

In [ ]:
df.head()

In [ ]:
sku_list = []
missings_list = []
data_list = []
for s in set(df['SKU'].tolist()):
    data = df[df['SKU'] == s]
    dates = pd.to_datetime(data['dates'])
    data.loc[:, ('dates')] = dates
    data = data.sort_values('dates')
    plt.plot(data['dates'], data['price_per_sku'].interpolate(), color='b')
    plt.plot(data['dates'], data['price_per_sku'].tolist(), color='r')
    plt.show()
    missings = len(data[data['price_per_sku'].isna()])
    sku_list.append(s)
    missings_list.append(missings)
    data.loc[:, 'price_per_sku'] = data['price_per_sku'].interpolate()
    data_list.append(data)
    
data = pd.DataFrame()
data['SKU'] = sku_list
data['nulls_count'] = missings_list
data.to_csv('first_sku_price_missings.csv', index=False)

### 3.1

In [ ]:
df = pd.read_csv('second_group_sku.csv')

In [ ]:
df.head()

In [ ]:
promo = pd.read_csv('promo_1510002.csv')

In [ ]:
promo.head()

In [ ]:
df = df.merge(promo, how='left', on='SKU')

In [ ]:
df.head()

In [ ]:
len(df[df['price_per_sku'].isna()])

In [ ]:
dates = pd.to_datetime(df['dates'])
df.loc[:, ('dates')] = dates

In [ ]:
for s in set(df['SKU'].tolist()):
    data = df[df['SKU'] == s]
    data = data.sort_values('dates')
    plt.plot(data['price_per_sku'], color='b')
    plt.show()
    break

In [ ]:
data.head()

In [ ]:
promo_count_list = []
sku_list = []
for s in set(promo['SKU'].tolist()):
    sku_list.append(s)
    data = promo[promo['SKU'] == s]
    promo_count_list.append(len(set(data['week_num'])))

In [ ]:
data = pd.DataFrame()
data['SKU'] = sku_list
data['promo_count'] = promo_count_list
data.to_csv('second_sku_promo_count.csv', index=False)

### 3.2

In [ ]:
import datetime

In [ ]:
df = pd.read_csv('second_group_sku.csv')

In [ ]:
promo = pd.read_csv('promo_1510002.csv')

In [ ]:
df = df.merge(promo, how='left', on='SKU')

In [ ]:
dates = pd.to_datetime(df['dates'])
df.loc[:, ('dates')] = dates

In [ ]:
pweek = df['dates'].dt.strftime('%U').tolist()

In [ ]:
pweek = [str(int(w)) for w in pweek]

In [ ]:
df['pweek'] = pweek

In [ ]:
df['pyear'] = df['dates'].dt.strftime('%Y')

In [ ]:
df.head()

In [ ]:
period = []
for y, w in zip(df['pyear'].tolist(), df['pweek'].tolist()):
    try:
        period.append(str(int(y)) + '-' + str(int(w)))
    except:
        period.append(-100)

In [ ]:
df['period'] = period

In [ ]:
promo_period = []
for y, w in zip(df['year'].tolist(), df['week_num'].tolist()):
    try:
        promo_period.append(str(int(y)) + '-' + str(int(w)))
    except:
        promo_period.append(-100)

In [ ]:
df['promo_period'] = promo_period

In [ ]:
sku_list = []
mean_num_purchases_promo_list = []
mean_num_purchases_wo_promo_list = []
for s in set(df['SKU'].tolist()):
    data = df[df['SKU'] == s]
    sku_list.append(s)
    promo_period = list(set(data['promo_period']))
    p = data[data['period'].isin(promo_period)][['dates', 'num_purchases']].drop_duplicates()['num_purchases'].mean()
    p_wo = data[~data['period'].isin(promo_period)][['dates', 'num_purchases']].drop_duplicates()['num_purchases'].mean()
    mean_num_purchases_promo_list.append(p)
    mean_num_purchases_wo_promo_list.append(p_wo)
    
    
data = pd.DataFrame()
data['SKU'] = sku_list
data['mean_num_purchases_promo']  = mean_num_purchases_promo_list
data['mean_num_purchases_wo_promo'] = mean_num_purchases_wo_promo_list

data.to_csv('second_sku_sales_comparison.csv', index=False)

### 4

In [ ]:
df = pd.read_csv('third_group_sku.csv')

In [ ]:
dates = pd.to_datetime(df['dates'])
df.loc[:, ('dates')] = dates

In [ ]:
df.head()

In [ ]:
conc_df = pd.read_csv('conc_df.csv')

In [ ]:
conc_df.head()

### 4.1

In [ ]:
for s in set(df['SKU'].tolist()):
    data = df[df['SKU'] == s]
    data = data.sort_values('dates')
    plt.plot(data['price_per_sku'], color='b')
    plt.show()

### 4.2

In [ ]:
sku_list = list(set(df['SKU'].tolist()))

In [ ]:
conc_rows_count_list = []

for s in sku_list:
    conc_rows_count_list.append(len(conc_df[conc_df['SKU'] == s]))

In [ ]:
data = pd.DataFrame()
data['SKU'] = sku_list
data['conc_rows_count'] = conc_rows_count_list
data.to_csv('third_sku_competitor_count.csv', index=False)

### 4.3

In [ ]:
# df = df.merge(conc_df, how='left', on='SKU')

In [ ]:
df = pd.read_csv('third_group_sku.csv')
conc_df = pd.read_csv('conc_df.csv')
dates = pd.to_datetime(df['dates'])
df.loc[:, ('dates')] = dates

In [ ]:
df['year'] = df['dates'].dt.strftime('%Y')

In [ ]:
pweek = df['dates'].dt.strftime('%U').tolist()

In [ ]:
pweek = [str(int(w)) for w in pweek]

In [ ]:
df['week'] = pweek

In [ ]:
period = []
for y, w in zip(df['year'].tolist(), df['week'].tolist()):
    try:
        period.append(str(int(y)) + '-' + str(int(w)))
    except:
        period.append(-100)

In [ ]:
df['period'] = period

In [ ]:
df = df.drop(['num_purchases', 'year', 'week'], axis=1).reset_index(drop=True)

In [ ]:
period = []
for y, w in zip(conc_df['year'].tolist(), conc_df['week_num'].tolist()):
    try:
        period.append(str(int(y)) + '-' + str(int(w)))
    except:
        period.append(-100)

In [ ]:
conc_df['period'] = period

In [ ]:
conc_df = conc_df.drop(['year', 'week_num', 'ret_net_code'], axis=1).reset_index(drop=True)

In [ ]:
data = df.merge(conc_df, how='left', on=('SKU', 'period'))

In [ ]:
data = data.dropna()

In [ ]:
data.head()

In [ ]:
sku_list = list(set(data['SKU']))

In [ ]:
lowest_price_rows_count = []
for sku in sku_list:
    df = data[data['SKU'] == sku]
    dates = list(set(df['dates']))
    cnt = 0
    for date in dates:
        min_price = df[df['dates'] == date]['price_per_sku_y'].min()
        cnt += len(df[(df['dates'] == date) & (df['price_per_sku_x'] <= min_price)])
    lowest_price_rows_count.append(cnt)

In [ ]:
data = pd.DataFrame()
data['SKU'] = sku_list
data['lowest_price_rows_count'] = lowest_price_rows_count
data.to_csv('third_sku_le_price.csv', index=False)

### 5

In [30]:
import pandas as pd
import matplotlib.pyplot as plt

In [31]:
df_test = pd.read_csv('test.csv')

In [32]:
dates = pd.to_datetime(df_test['dates'])
df_test.loc[:, ('dates')] = dates

In [33]:
sku_list = list(set(df_test['SKU'].tolist()))

In [34]:
files = [
    'first_group_sku.csv',
    'second_group_sku.csv',
    'third_group_sku.csv'
]

In [35]:
df_list = []
for file in files:
    df = pd.read_csv(file)
    df = df[df['SKU'].isin(sku_list)]
    df_list.append(df)

In [36]:
df = pd.concat(df_list).sample(frac=1).reset_index(drop=True)

In [37]:
dates = pd.to_datetime(df['dates'])
df.loc[:, ('dates')] = dates

In [74]:
# pip install neuralprophet --trusted-host pypi.org --trusted-host pypi.python.org

In [75]:
from neuralprophet import NeuralProphet

In [85]:
from tqdm import tqdm

In [90]:
df_list = []
for sku in tqdm(sku_list):
    data = df[df['SKU'] == sku]
    data = data.sort_values('dates').reset_index(drop=True)

    df0 = pd.DataFrame()
    df0['ds'] = data['dates']
    df0['y'] = data['price_per_sku']
    
    data_test = df_test[df_test['SKU'] == sku].reset_index(drop=True)
    
    df1 = pd.DataFrame()
    df1['ds'] = data_test['dates']
    df1['y'] = None
    
    m = NeuralProphet()
    metrics = m.fit(df0, freq="D")
    forecast = m.predict(df1)
    
    df2 = pd.DataFrame()
    df2['dates'] = data_test['dates']
    df2['SKU'] = data_test['SKU']
    df2['price_per_sku'] = forecast['yhat1']
    
    df_list.append(df2)



  0%|          | 0/25 [00:00<?, ?it/s]INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 73.684% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 556
WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (6) is too small than the required number for the learning rate finder (207). The results might not be optimal.


Finding best initial lr:   0%|          | 0/207 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 85.714% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 85.714% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D


Predicting: 6it [00:00, ?it/s]

...

Finding best initial lr:   0%|          | 0/212 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 85.714% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 85.714% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D


Predicting: 14it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


100%|██████████| 25/25 [07:39<00:00, 18.38s/it]


In [92]:
df = pd.concat(df_list)
df.to_csv('prediction.csv', index=False)